In [ ]:
# Personally I had to add the root folder of the repo to the sys.path.  If certain imports do not work you should uncomment and set the following.
# import sys
# sys.path.append('/root/of/repo/folder/')

# Edge Node graph from OpenStreetMaps

To create a map of edge nodes that somewhat resembles population density and shows clusters around dense areas we want to use the national train connections.  We make use of the [OpenStreetMap Data](https://download.geofabrik.de) and use the [osmium-tool](https://osmcode.org/osmium-tool/manual.html) to query the data.  To interface with the file from Python we use [PyOsmium](https://docs.osmcode.org/pyosmium/latest/).

We use the `StationHandler` to extract the `nodes` and `ways` in the dataset that represent `stop_position`s for trains and `railway`s, respectively.

In [ ]:
import osmium

osm_file = "./out/netherlands-latest.osm.bz2"

In [ ]:
class StationHandler(osmium.SimpleHandler):
    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.stops = {}
        self.stations = {}
        self.track = []
    
    def node(self, n):
        # We filter on n\train=yes to get only train related nodes, additionally we found that stations are the only items with n\wikidata=.
        if n.tags.get('train') == 'yes':
            if n.tags.get('public_transport') == "stop_position":
                self.stops[str(n.id)] = dict(n.tags)
            if n.tags.get("wikidata", "") != "":
                self.stations[str(n.id)] = dict(n.tags)

    def way(self, w):
        # We filter on w\railway=rail
        if w.tags.get('railway') == 'rail':
            self.track.append([ str(x) for x in w.nodes ])

    def relation(self, r):
        pass

In [ ]:
s = StationHandler()
s.apply_file(osm_file)

## TrainData

As the extracting from the full-sized dataset can take a long time (on my machine it took about 30 minutes for the NL dataset) we want to save the extracted data.  This is done by storing the data in a `TrainData` object that can be pickled.

In [ ]:
from dataclasses import dataclass

@dataclass
class TrainData:
    stops: dict[str, dict[str, any]]
    stations: dict[str, dict[str, any]]
    track: list[list[str]]

In [ ]:
train_data = TrainData(stops=s.stops, stations=s.stations, track=s.track)

In [ ]:
import pickle
pickle_file = "nl-train-data.pickle"

In [ ]:
# with open(pickle_file, 'wb') as f:
    # pickle.dump(train_data, f)

In [ ]:
import pickle
with open(pickle_file, 'rb') as f:
    train_data = pickle.load(f)

## Processing

Our goal is to create a graph where each vertex is a station and the edges represent train track between them.  Each station can have different stop positions.  For this reason we make use of the name of a stop position as they are the same for all stop positions at a station.

To create our graph we need to follow different pieces of track (ways) from one stop position until we find a different stop position.  This process has been implemented in a recursive function with a limit of `10000` on the recursion depth.

In [ ]:
station_stops = set(train_data.stops.keys())
stations_by_ele = { v['name']: (k,v) for k, v in train_data.stations.items() }

In [ ]:
from collections import defaultdict

graph = defaultdict(set)
for track in train_data.track:
    prev_node = None
    for node in track:
        if prev_node:
            graph[prev_node].add(node)
            graph[node].add(prev_node)
        prev_node = node

In [ ]:
from functools import reduce
import sys
sys.setrecursionlimit(10000)

def get_station_at(loc):
    stop = train_data.stops.get(loc)
    if stop != None:
        return stop.get('name', 'No name')
    return None

def find_next_station(station, loc, visited, depth):
    """Recursive function to find the next station (stop position) on the line.
    
    Is limited by the recursionlimit to avoid killing the kernel.
    """
    if depth >= sys.getrecursionlimit():
        return []
    visited.add(loc)
    st = get_station_at(loc)
    if st != None and st != station:
        return [ st ]

    outgoing_track = [ x for x in graph[loc] if x not in visited ]
    if len(outgoing_track) == 0:
        return []

    return reduce(lambda a,b:a+b, [ find_next_station(station, x, visited, depth + 1) for x in outgoing_track ], [])

In [ ]:
station_graph = defaultdict(set)
for stop in train_data.stops.keys():
    station = get_station_at(stop)
    station_graph[station].update(find_next_station(station, stop, set(), 0))

## Station Graph

Here we save the station graph to file such that we do not need to re-run the logic above a second time.

In [ ]:
import json

# with open('graph-nl.json', 'w') as f:
#     f.write(json.dumps({ name: list(values) for name, values in station_graph.items() }))

with open('graph-nl.json', 'r') as f:
    station_graph = defaultdict(set)
    loaded_graph = json.loads(f.read())
    for name, connections in loaded_graph.items():
        if len(connections) > 0:
            station_graph[name] = set(connections)

print(f"Found {len(station_graph)} stations!")

## Visualisation

As a quick debugging tool we make use of `graphviz` to visualise the extracted graph.

In [ ]:
import graphviz
from collections import defaultdict

def visualise(graph, name):
    g = graphviz.Graph('G', filename=f"{name}.gv", engine='sfdp')

    connected = defaultdict(set)
    def is_connected(start, end):
        return end in connected[start] or start in connected[end]

    for station, linked_stations in graph.items():
        for l in linked_stations:
            if not is_connected(station, l) and station != l:
                g.edge(station, l)
                connected[station].add(l)

    g.view()

## Different Setups

Now that we have a graph of stations and tracks between them we need to create different setups that represent more centralised setups.  To do this we define a function that correctly updates the graph while grouping a `nodeB` with `nodeA`.

In [ ]:
def group_nodes(graph, nodeA, nodeB):
    """Group two nodes A and B together into node A.

    Delete node B, join all outgoing links from B with A, and update all 
    incoming links for B to A.
    """
    outgoing_links = [ x for x in graph[nodeB] if x != nodeB ]
    del graph[nodeB]
    graph[nodeA] = (graph[nodeA] | set(outgoing_links)) - { nodeA, nodeB }
    for node, values in graph.items():
        if nodeB in values:
            graph[nodeA].add(node)
            graph[node] = (graph[node] - { nodeB }) | { nodeA }

In the next block we define the main function that creates an alternate graph with a target in terms of number of nodes for the graph.  Over several iterations the function will decrease the graph by grouping the least occuring node with one of its neighbours.  The least occuring node is defined as the node that is still part of the graph (has not been grouped into a different node) and has the least amount of nodes pointing to it.  A node points to a different node when it is grouped into that different node.

This strategy allows us to create a reasonably uniform distribution of mapped nodes.  This is important for the Hybrid strategies as a uniform distribution of nodes is important to get similar performance over the different groupings.

In [ ]:
import random

def count_occurances(mapping) -> dict[str, int]:
    """Count the occurances of target nodes in the mapping."""
    counter = defaultdict(int)
    for value in mapping.values():
        counter[value] += 1
    return counter

def get_least_occuring_node(mapping: dict[str, str]) -> str:
    """Return a node that occurs the least in the mapping.

    If there are multiple nodes that occur the least a random node is selected.
    """
    occurances = count_occurances(mapping)
    items = list(occurances.items())
    random.shuffle(items)
    return min(items, key=lambda x: x[1])[0]

def create_mapping(mapping, from_node, to_node):
    """Create a mapping `from` to a different node.
    
    Updates the maping by inserting the new mapping and updating all existing 
    mappings that used to point to `from_node` to `to_node`.
    """
    mapping[from_node] = to_node
    for node, to in mapping.items():
        if to == from_node:
            mapping[node] = to_node

def create_alternate_graph(original_graph, no_nodes: int):
    alternate_graph = dict.copy(original_graph)
    mapping = { node: node for node in alternate_graph.keys() }
    while len(alternate_graph) > no_nodes:
        main_node = get_least_occuring_node(mapping)
        neighbouring_nodes = list(alternate_graph[main_node])
        if len(neighbouring_nodes) == 0:
            print(f"No outgoing links: {main_node}")
            continue
        neighbouring_node = random.choice(list(alternate_graph[main_node]))
        group_nodes(alternate_graph, main_node, neighbouring_node)
        create_mapping(mapping, neighbouring_node, main_node)
    return alternate_graph, mapping

The block below is used for quickly testing the grouping strategy and visualising the result.

In [ ]:
alternate_graph, mapping = create_alternate_graph(station_graph, 256)
print(f"Found {len(set(mapping.values()))} nodes in the mapping:")
print(json.dumps(count_occurances(mapping), indent=4))
print(f"Found {len(set(alternate_graph.keys()))} nodes in the graph:")
print(json.dumps({ node: list(values) for node, values in alternate_graph.items() }, indent=4))
visualise(alternate_graph, "test")

## Setups
To test the effect of differently sized edge networks we want to create a subset of sizes that we will use.

In [ ]:
max_nodes = len(station_graph)
setups = []
no_nodes = 1
while no_nodes < max_nodes:
    setups.append(no_nodes)
    no_nodes = no_nodes * 2
setups.append(max_nodes)
print(setups)

In [ ]:
def clear_names(graph):
    """Sets more generic names (cdn1, cdn2, ..., cdnN) for the nodes in the graph."""
    name_map = { name: f"cdn{n}" for n, name in enumerate(graph.keys()) }
    return { name_map[name]: set([ name_map[v] for v in values ]) for name, values in graph.items() }

In [ ]:
generic_graph = clear_names(station_graph)

for size in setups:
    alternate_graph, mapping = create_alternate_graph(generic_graph, size)
    visualise(alternate_graph, str(size))
    with open(f"./out/setups/graph-{size}.json", 'w') as f:
        json.dump({ node: list(values) for node, values in alternate_graph.items()}, f)
    with open(f"./out/setups/mapping-{size}.json", 'w') as f:
        json.dump(mapping, f)